In [1]:
%reload_ext autoreload
%autoreload 2

import vtk
from vtk.util import numpy_support

import numpy as np
import bloscpack as bp

import sys
import os
sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from annotation_utilities import *
from metadata import *

from skimage.measure import mesh_surface_area, marching_cubes, correct_mesh_orientation

import time

import matplotlib.pyplot as plt
%matplotlib inline

from vis3d_utilities import *

Setting environment for Local Macbook Pro


/usr/lib/python2.7/dist-packages/pkg_resources.py:1031: UserWarning: /home/yuncong/.python-eggs is writable by group/others and vulnerable to attack when used with get_resource_filename. Consider a more secure location (set with .set_extraction_path or the PYTHON_EGG_CACHE environment variable).
  warnings.warn(msg, UserWarning)


In [2]:
with open('colors.txt', 'r') as f:
    colors = {l[0]: np.r_[float(l[1]), float(l[2]), float(l[3])] for l in map(lambda x: x.split(), f.readlines())}

In [3]:
stack = 'MD589'

In [7]:
# Load
polydata_list = {}

for name_s in labels_sided:

    fn = mesh_rootdir + "/%(stack)s/%(stack)s_%(name)s_smoothed.stl" % {'stack': stack, 'name': name_s}

    reader = vtk.vtkSTLReader()
    reader.SetFileName(fn)
    reader.Update()

    polydata = reader.GetOutput()    
    polydata_list[name_s] = polydata

In [8]:
ren1 = vtk.vtkRenderer()

renWin1 = vtk.vtkRenderWindow()
renWin1.AddRenderer(ren1)

iren1 = vtk.vtkRenderWindowInteractor()
iren1.SetRenderWindow(renWin1)

camera = vtk.vtkCamera()

# 45 degree
camera.SetViewUp(0, -1, 0)
camera.SetPosition(-20, -30, -10)
camera.SetFocalPoint(1, 1, 1)

# # saggital
# camera.SetViewUp(0, -1, 0)
# camera.SetPosition(0, 0, -2)
# camera.SetFocalPoint(0, 0, 1)

# # coronal
# camera.SetViewUp(0, -1, 0)
# camera.SetPosition(-2, 0, 0)
# camera.SetFocalPoint(-1, 0, 0)

# # horizontal
# camera.SetViewUp(0, 0, -1)
# camera.SetPosition(0, 1, 0)
# camera.SetFocalPoint(0, -1, 0)

# for i, (l, polydata) in enumerate(polydata_list.iteritems()):
for i, (name_s, polydata) in enumerate(polydata_list.iteritems()):

    if name_s == 'outerContour' or name_s == 'sp5':
        continue
    
    m = vtk.vtkPolyDataMapper()
    m.SetInputData(polydata)
    #     m.SetInputData(reader.GetOutput())
    #     m.SetInputConnection(reader.GetOutputPort())

    a = vtk.vtkActor()
    a.SetMapper(m)
    a.GetProperty().SetRepresentationToWireframe()
    a.GetProperty().SetColor(colors[labelMap_sidedToUnsided[name_s]])

    if name_s == 'outerContour':
        a.GetProperty().SetOpacity(.1)
            
    ren1.AddActor(a)

ren1.SetActiveCamera(camera)

widget1 = add_axes(iren1)

renWin1.Render()

ren1.ResetCamera()

renWin1.SetWindowName('original')

iren1.Start()

In [9]:
# Load
polydata_gaussianSmoothed_list = {}

for name_s in labels_sided:

    fn = mesh_rootdir + "/%(stack)s/%(stack)s_%(name)s_gaussianSmoothed.stl" % {'stack': stack, 'name': name_s}

    reader = vtk.vtkSTLReader()
    reader.SetFileName(fn)
    reader.Update()

    polydata = reader.GetOutput()    
    polydata_gaussianSmoothed_list[name_s] = polydata

In [10]:
ren1 = vtk.vtkRenderer()
ren2 = vtk.vtkRenderer()

renWin1 = vtk.vtkRenderWindow()
renWin1.AddRenderer(ren1)

renWin2 = vtk.vtkRenderWindow()
renWin2.AddRenderer(ren2)

iren1 = vtk.vtkRenderWindowInteractor()
iren1.SetRenderWindow(renWin1)

iren2 = vtk.vtkRenderWindowInteractor()
iren2.SetRenderWindow(renWin2)

camera = vtk.vtkCamera()

# 45 degree
camera.SetViewUp(0, -1, 0)
camera.SetPosition(-20, -30, -10)
camera.SetFocalPoint(1, 1, 1)

# # saggital
# camera.SetViewUp(0, -1, 0)
# camera.SetPosition(0, 0, -2)
# camera.SetFocalPoint(0, 0, 1)

# # coronal
# camera.SetViewUp(0, -1, 0)
# camera.SetPosition(-2, 0, 0)
# camera.SetFocalPoint(-1, 0, 0)

# # horizontal
# camera.SetViewUp(0, 0, -1)
# camera.SetPosition(0, 1, 0)
# camera.SetFocalPoint(0, -1, 0)

# for i, (l, polydata) in enumerate(polydata_list.iteritems()):
for i, (name_s, polydata) in enumerate(polydata_gaussianSmoothed_list.iteritems()):

    if name_s == 'outerContour' or name_s == 'sp5':
        continue
    
    m = vtk.vtkPolyDataMapper()
    m.SetInputData(polydata)
    #     m.SetInputData(reader.GetOutput())
    #     m.SetInputConnection(reader.GetOutputPort())

    a = vtk.vtkActor()
    a.SetMapper(m)
    a.GetProperty().SetRepresentationToWireframe()
    a.GetProperty().SetColor(colors[labelMap_sidedToUnsided[name_s]])

    if name_s == 'outerContour':
        a.GetProperty().SetOpacity(.1)
            
    ren1.AddActor(a)

ren1.SetActiveCamera(camera)

for i, (name_s, polydata) in enumerate(polydata_list.iteritems()):

    if name_s == 'outerContour' or name_s == 'sp5':
        continue
    
    m = vtk.vtkPolyDataMapper()
    m.SetInputData(polydata)
    #     m.SetInputData(reader.GetOutput())
    #     m.SetInputConnection(reader.GetOutputPort())

    a = vtk.vtkActor()
    a.SetMapper(m)
#     a.GetProperty().SetRepresentationToWireframe()
    a.GetProperty().SetColor(colors[labelMap_sidedToUnsided[name_s]])
    
    if name_s == 'outerContour':
        a.GetProperty().SetOpacity(.1)

    ren2.AddActor(a)

ren2.SetActiveCamera(camera)

widget1 = add_axes(iren1)
widget2 = add_axes(iren2)

renWin1.Render()
renWin2.Render()

ren1.ResetCamera()
ren2.ResetCamera()

# ren1.SetBackground(1,1,1)
# ren2.SetBackground(1,1,1)

renWin1.SetWindowName('smoothed')
renWin2.SetWindowName('original')

iren1.Start()

In [ ]:
# Load shifted normalized/symmetric landmarks
shifted_poly_allLandmarks = {}

for name_s in labels_sided:

    fn = mesh_rootdir + "/%(name)s_average_shifted_icp.stl" % {'stack': 'MD589', 'name': name_s}

    reader = vtk.vtkSTLReader()
    reader.SetFileName(fn)
    reader.Update()

    polydata = reader.GetOutput()    
    shifted_poly_allLandmarks[name_s] = polydata

In [ ]:
# colors = {name: np.random.randint(0, 255, (3,)) / 255. for name in labels_unsided}
# colors['outerContour'] = np.r_[1.,1.,1.]

# with open('colors.txt', 'w') as f:
#     for name, c in colors.iteritems():
#         f.write('%s %f %f %f\n' % (name, c[0], c[1], c[2]))

In [ ]:
# if os.path.exists('colors.txt'):
#     colors = np.loadtxt('colors.txt')    
# else:
#     colors = np.random.randint(0, 255, (100, 3)) / 255.
#     np.savetxt('colors.txt', colors)

ren1 = vtk.vtkRenderer()
renWin1 = vtk.vtkRenderWindow()
renWin1.AddRenderer(ren1)
iren1 = vtk.vtkRenderWindowInteractor()
iren1.SetRenderWindow(renWin1)

for name, polydata in shifted_poly_allLandmarks.iteritems():
    
#     if name == 'outerContour':
#         continue
    
    m = vtk.vtkPolyDataMapper()
    m.SetInputData(polydata)

    a = vtk.vtkActor()
    a.SetMapper(m)
#     a.GetProperty().SetRepresentationToWireframe()
    a.GetProperty().SetColor(colors[labelMap_sidedToUnsided[name]])
    
    if name == 'outerContour':
        a.GetProperty().SetOpacity(.1)
        
    ren1.AddActor(a)

axes_widget1 = add_axes(iren1)

renWin1.Render()
renWin1.SetWindowName('atlas')

camera = vtk.vtkCamera()

# 45 degree
camera.SetViewUp(0, -1, 0)
camera.SetPosition(-20, -30, -10)
camera.SetFocalPoint(1, 1, 1)

# # saggital
# camera.SetViewUp(0, -1, 0)
# camera.SetPosition(0, 0, -2)
# camera.SetFocalPoint(0, 0, 1)

# # coronal
# camera.SetViewUp(0, -1, 0)
# camera.SetPosition(-2, 0, 0)
# camera.SetFocalPoint(-1, 0, 0)

# # horizontal
# camera.SetViewUp(0, 0, 1)
# camera.SetPosition(0, -1, 0)
# camera.SetFocalPoint(0, 1, 0)

ren1.SetActiveCamera(camera)

ren1.ResetCamera()
iren1.Start()